# Build a simulation using the Python API

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

Currently, this notebook manipulates the simulation by directly placing objects inside of the attributes of the network and domain. It should be refactored when proper methods exist for adding these objects.

Import the Simulation class

In [1]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [2]:
my_sim = Simulation()
net = my_sim.network
my_sim.describe_state()

{'uuid': 'cfa100b7-7970-4370-a74d-8e5268db25ff',
 'network': {'uuid': 'ddef284e-6c84-4a11-b398-794d55d1177b',
  'nodes': {},
  'links': {}},
 'domain': {'uuid': '5ef25901-084d-42a3-b623-5126f996be2a', 'accounts': {}}}

## Add nodes

In [3]:
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server

In [4]:
my_pc = Computer.from_config(
    config={
        "type": "computer",
        "hostname":"pc_1",
        "ip_address":"192.168.1.10",
        "subnet_mask":"255.255.255.0",
    }
   )
net.add_node(my_pc)
my_server = Server.from_config(
    config={
        "type": "server",
        "hostname":"Server",
        "ip_address":"192.168.1.11",
        "subnet_mask":"255.255.255.0"
    }
)
net.add_node(my_server)


## Connect the nodes

In [5]:
from primaite.simulator.network.hardware.nodes.host.host_node import NIC
from primaite.simulator.network.hardware.nodes.network.switch import Switch


In [6]:
my_switch = Switch.from_config(
    config = {
        "type":"switch",
        "hostname":"switch1",
        "num_ports":12
    }
)
net.add_node(my_switch)

pc_nic = NIC(ip_address="130.1.1.1", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_pc.connect_nic(pc_nic)

server_nic = NIC(ip_address="130.1.1.2", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_server.connect_nic(server_nic)

net.connect(pc_nic, my_switch.network_interface[1])
net.connect(server_nic, my_switch.network_interface[2])


Link(uuid='6a5d9dc3-dab2-413b-a709-fa3c9d218b53', endpoint_a=NIC(ip_address=IPv4Address('130.1.1.2'), subnet_mask=IPv4Address('255.255.255.0'), uuid='86d99ed3-c0b8-4620-b309-25595acb817b', mac_address='e0:b4:28:49:f5:14', speed=100.0, mtu=1500, enabled=True, port_num=2, port_name=None, pcap=<primaite.simulator.system.core.packet_capture.PacketCapture object at 0x7fdcde7a49a0>, nmne={}, traffic={}, wake_on_lan=False, gateway='130.1.1.255'), endpoint_b=SwitchPort(uuid='fff219e1-3404-4a50-89a9-7afde9da1558', mac_address='75:b7:d6:ba:9a:b5', speed=100.0, mtu=1500, enabled=True, port_num=2, port_name=None, pcap=<primaite.simulator.system.core.packet_capture.PacketCapture object at 0x7fdcde7a4af0>, nmne={}, traffic={}), bandwidth=100.0, current_load=0.0)

## Add files and folders to nodes


In [7]:
from primaite.simulator.file_system.file_type import FileType
from primaite.simulator.file_system.file_system import File
from primaite.simulator.system.core.sys_log import SysLog

In [8]:
my_pc_downloads_folder = my_pc.file_system.create_folder("downloads")
my_pc_downloads_folder.add_file(File(name="firefox_installer.zip",folder_id="Test", folder_name="downloads" ,file_type=FileType.ZIP, sys_log=SysLog(hostname="Test")))

In [9]:
my_server_folder = my_server.file_system.create_folder("static")
my_server.file_system.create_file("favicon.ico", file_type=FileType.PNG)

File(uuid='7dddaccf-0277-4442-b8a8-54fcf546f631', name='favicon.ico', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.NONE: 0>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x7fdcdfd93fd0>, deleted=False, folder_id='17a128b7-66a4-497a-ac67-3e46dc32de16', folder_name='root', file_type=<FileType.UNKNOWN: 0>, sim_size=0, num_access=0, sim_root=PosixPath('/home/runner/primaite/4.0.0/sessions/2025-03-24/10-01-47/Server/fs'), folder=Folder(uuid='17a128b7-66a4-497a-ac67-3e46dc32de16', name='root', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.NONE: 0>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x7fdcdfd93fd0>, deleted=False, files={'7dddaccf-0277-4442-b8a8-54fcf546f631': File(uuid='7dddaccf-0277-4442-b8a8-54fcf546f631', name='favicon.ico', health_status=<Fil

## Add applications to nodes

In [10]:
from pydantic import Field

from pathlib import Path
from primaite.simulator.system.applications.application import Application, ApplicationOperatingState
from primaite.simulator.system.software import SoftwareHealthState, SoftwareCriticality
from primaite.simulator.file_system.file_system import FileSystem
from primaite.utils.validation.ip_protocol import PROTOCOL_LOOKUP
from primaite.utils.validation.port import PORT_LOOKUP
from primaite.simulator.system.core.sys_log import SysLog


# no applications exist yet so we will create our own.
class MSPaint(Application, discriminator="MSPaint"):
    class ConfigSchema(Application.ConfigSchema):
        type: str = "MSPaint"

    config: ConfigSchema = Field(default_factory=lambda: MSPaint.ConfigSchema())

    def __init__(self, **kwargs):
        kwargs["name"] = "MSPaint"
        kwargs["port"] = PORT_LOOKUP["HTTP"]
        kwargs["protocol"] = PROTOCOL_LOOKUP["NONE"]
        super().__init__(**kwargs)

    def describe_state(self):
        return super().describe_state()

In [11]:
my_pc.software_manager.install(MSPaint)
mspaint = my_pc.software_manager.software.get("MSPaint")

In [12]:
my_pc.applications[mspaint.uuid] = mspaint

## Create a domain account

In [13]:
from primaite.simulator.domain.account import Account, AccountType


In [14]:
acct = Account(username="admin", password="admin12", account_type=AccountType.USER)
my_sim.domain.accounts[acct.uuid] = acct

Verify that the state dictionary contains no non-serialisable objects.

In [15]:
my_sim.describe_state()

{'uuid': 'cfa100b7-7970-4370-a74d-8e5268db25ff',
 'network': {'uuid': 'ddef284e-6c84-4a11-b398-794d55d1177b',
  'nodes': {'pc_1': {'uuid': '0bab65e8-36a1-4a54-86c4-84b69e27bc34',
    'hostname': 'pc_1',
    'operating_state': 1,
    'NICs': {1: {'uuid': '083c901e-753c-4e3e-a829-710e993d54a2',
      'mac_address': 'f3:38:b8:0f:94:77',
      'speed': 100.0,
      'mtu': 1500,
      'enabled': False,
      'traffic': {},
      'ip_address': '192.168.1.10',
      'subnet_mask': '255.255.255.0',
      'wake_on_lan': False},
     2: {'uuid': '56518a14-9043-4f7f-8b35-0bd402ef829d',
      'mac_address': '0a:7b:01:71:ac:ee',
      'speed': 100.0,
      'mtu': 1500,
      'enabled': True,
      'traffic': {},
      'ip_address': '130.1.1.1',
      'subnet_mask': '255.255.255.0',
      'wake_on_lan': False}},
    'file_system': {'uuid': '835b066c-0ee3-4979-901d-e65369d6f721',
     'folders': {'root': {'uuid': 'efc0c3df-f942-49a1-bc76-95a8eb6b579e',
       'name': 'root',
       'health_status': 1

In [16]:
import json
json.dumps(my_sim.describe_state())

'{"uuid": "cfa100b7-7970-4370-a74d-8e5268db25ff", "network": {"uuid": "ddef284e-6c84-4a11-b398-794d55d1177b", "nodes": {"pc_1": {"uuid": "0bab65e8-36a1-4a54-86c4-84b69e27bc34", "hostname": "pc_1", "operating_state": 1, "NICs": {"1": {"uuid": "083c901e-753c-4e3e-a829-710e993d54a2", "mac_address": "f3:38:b8:0f:94:77", "speed": 100.0, "mtu": 1500, "enabled": false, "traffic": {}, "ip_address": "192.168.1.10", "subnet_mask": "255.255.255.0", "wake_on_lan": false}, "2": {"uuid": "56518a14-9043-4f7f-8b35-0bd402ef829d", "mac_address": "0a:7b:01:71:ac:ee", "speed": 100.0, "mtu": 1500, "enabled": true, "traffic": {}, "ip_address": "130.1.1.1", "subnet_mask": "255.255.255.0", "wake_on_lan": false}}, "file_system": {"uuid": "835b066c-0ee3-4979-901d-e65369d6f721", "folders": {"root": {"uuid": "efc0c3df-f942-49a1-bc76-95a8eb6b579e", "name": "root", "health_status": 1, "visible_status": 0, "previous_hash": null, "revealed_to_red": false, "files": {}, "deleted_files": {}, "scanned_this_step": false},